In [19]:
import sys, os
print(sys.version)
print(os.environ['HOME'])
java_path = '/home/jovyan/conda'
os.environ['JAVA_HOME'] = java_path
print(os.environ.get('JAVA_HOME'))

3.5.5 | packaged by conda-forge | (default, Jul 23 2018, 23:45:43) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
/home/jovyan
/home/jovyan/conda


In [20]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [22]:
from pyspark.dbutils import DBUtils

dbutils = DBUtils(spark.sparkContext)
dbutils.fs.ls("/mnt")

[FileInfo(path='dbfs:/mnt/AWStest/', name='AWStest/', size=0),
 FileInfo(path='dbfs:/mnt/WoS/', name='WoS/', size=0),
 FileInfo(path='dbfs:/mnt/test/', name='test/', size=0),
 FileInfo(path='dbfs:/mnt/test2/', name='test2/', size=0),
 FileInfo(path='dbfs:/mnt/test3/', name='test3/', size=0),
 FileInfo(path='dbfs:/mnt/test7/', name='test7/', size=0)]

In [1]:
import pandas as pd
ego = pd.read_csv('../../packages/issi_data_package/output_files/data/774e7eb6-6ac7-4dd5-9339-531b746cb8ec.csv')
degree1 = pd.read_csv('../../packages/issi_data_package/output_files/data/774e7eb6-6ac7-4dd5-9339-531b746cb8ec_nodes.csv')
#display(ego,degree1)

In [9]:
edges = pd.read_csv('../../packages/issi_data_package/output_files/data/a71d68fa-a479-47b6-8e55-ce8f12eacb56_edges.csv')
citations = pd.merge(edges, degree1, left_on=['To'], right_on=['paper_id'])
citations.count()
#edges.count()

From                      413898
To                        413898
paper_id                  413898
date                      413898
journal_id                315637
citation_count            413898
original_title            413898
issue                     291890
paper_title               413898
year                      413898
first_name                341519
last_name                 325209
original_venue            364573
doc_type                  338453
volume                    317484
estimated_citation        413898
conference_instance_id      6265
book_title                 15357
rank                      413898
publisher                 363905
created_date              413898
reference_count           413898
conference_series_id       21136
doi                       351035
dtype: int64

In [10]:
citations0 = citations[['From','To']]
citations0.columns = ['focal', 'connector']
citations2 = pd.merge(citations0, citations, left_on=['connector'], right_on=['To'])
citations2.count()

focal                     11464864
connector                 11464864
From                      11464864
To                        11464864
paper_id                  11464864
date                      11464864
journal_id                 8825391
citation_count            11464864
original_title            11464864
issue                      8202022
paper_title               11464864
year                      11464864
first_name                 9345237
last_name                  9001069
original_venue             9749807
doc_type                   9305687
volume                     8716740
estimated_citation        11464864
conference_instance_id       69895
book_title                  234915
rank                      11464864
publisher                 10096385
created_date              11464864
reference_count           11464864
conference_series_id        421470
doi                        9712101
dtype: int64

In [11]:
citations00 = citations0
citations00.columns = ['cited2','LE']
citations3 = pd.merge(citations2, citations00, left_on=['focal','To'], right_on=['cited2','LE'])
citations3.count()

focal                     17989764
connector                 17989764
From                      17989764
To                        17989764
paper_id                  17989764
date                      17989764
journal_id                13919971
citation_count            17989764
original_title            17989764
issue                     13006668
paper_title               17989764
year                      17989764
first_name                14728317
last_name                 14219537
original_venue            15319381
doc_type                  14663909
volume                    13747072
estimated_citation        17989764
conference_instance_id      107713
book_title                  364885
rank                      17989764
publisher                 15824819
created_date              17989764
reference_count           17989764
conference_series_id        651528
doi                       15262445
cited2                    17989764
LE                        17989764
dtype: int64

In [18]:
# direct citations
import matplotlib
import matplotlib.pyplot as plt
from collections import defaultdict

paper_citing = defaultdict(list)

citingFile = open("../../packages/issi_data_package/output_files/data/a71d68fa-a479-47b6-8e55-ce8f12eacb56_edges.csv", "r")
citingFile.readline()
for line in citingFile:
    line = line.strip().replace('"', '').split(",")
    paper_citing[int(line[1])].append(int(line[0]))

In [26]:
# indirect citations
count = 0
total = len(degree1)
indirect_citation = defaultdict(list)
paperList = degree1['paper_id'].tolist()

for paper in paperList:
    count += 1
    if count % 10000 == 0:
        print("Processed " + str(float(count) / float(total)) + " of total focal papers...\n")
    for citing_paper_1 in paper_citing[paper]:
        for citing_paper_2 in paper_citing[paper]:
            if citing_paper_1 in paper_citing[citing_paper_2]:
                temp = []
                temp.append(citing_paper_1)
                temp.append(citing_paper_2)
                indirect_citation[paper].append(temp)

Processed 0.25060772373004536 of total focal papers...

Processed 0.5012154474600907 of total focal papers...

Processed 0.7518231711901361 of total focal papers...



In [30]:
import pyspark.sql.functions as sql
papers = spark.read.format("parquet").load("/mnt/test/graph/2019-1-11-PNAS/papers/*.parquet")
papers.describe().filter(sql.col("summary") == "count").show(20,False)

+-------+------+------+------+------+------+------+---+------+------+------+------+----+----+------+------+------+------+------+------+------+------+------+
|summary|_c0   |_c1   |_c2   |_c3   |_c4   |_c5   |_c6|_c7   |_c8   |_c9   |_c10  |_c11|_c12|_c13  |_c14  |_c15  |_c16  |_c17  |_c18  |_c19  |_c20  |_c21  |
+-------+------+------+------+------+------+------+---+------+------+------+------+----+----+------+------+------+------+------+------+------+------+------+
|count  |138953|138953|135508|138953|138953|138953|25 |138953|138953|135368|138953|0   |0   |135531|135099|133690|127181|138953|138953|138953|138953|138953|
+-------+------+------+------+------+------+------+---+------+------+------+------+----+----+------+------+------+------+------+------+------+------+------+



In [11]:
help('modules')


Please wait a moment while I gather a list of all available modules...



/home/jovyan/conda/dbconnect/lib/python3.5/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/home/jovyan/conda/dbconnect/lib/python3.5/pkgutil.py:104: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    
  yield from walk_packages(path, name+'.', onerror)


CDROM               autoreload          jsonschema          select
DLFCN               backcall            jupyter             selectors
IN                  base64              jupyter_client      send2trash
IPython             bdb                 jupyter_core        setuptools
PyQt5               binascii            keyword             shelve
TYPES               binhex              kiwisolver          shlex
__future__          bisect              lib2to3             shutil
_ast                bleach              linecache           signal
_bisect             builtins            locale              simplegeneric
_bootlocale         bz2                 logging             sip
_bz2                cProfile            lzma                sipconfig
_codecs             calendar            macpath             sipdistutils
_codecs_cn          certifi             macurl2path         site
_codecs_hk          cgi                 mailbox             six
_codecs_iso2022     cgitb               mail